# 🎬 Voice AI Demo - Interactive TTS Interface
## Text-to-Speech Inference with Gradio

This notebook provides:
1. ✅ Load fine-tuned TTS model
2. ✅ Interactive Gradio web interface
3. ✅ A/B comparison (base vs fine-tuned)
4. ✅ Audio playback and download
5. ✅ Optional HuggingFace upload

**⚠️ Important:** Ensure setup.ipynb and train_or_finetune.ipynb have been run first!

**📝 Note:** This demo uses Python 3.12 compatible TTS (coqui-tts from Idiap Research Institute).

## Step 1: Import Libraries and Setup

In [ ]:
import os
import sys
import torch
import gradio as gr
import numpy as np
import librosa
import soundfile as sf
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("🔧 Importing libraries...")

# Setup paths
BASE_DIR = "/content/voiceai"
DRIVE_DIR = "/content/drive/MyDrive/voiceai"
CHECKPOINT_DIR = f"{DRIVE_DIR}/checkpoints"
OUTPUT_DIR = f"{DRIVE_DIR}/outputs"

# Ensure directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU detected. Inference will use CPU.")

print(f"\n📂 Paths:")
print(f"   • Checkpoints: {CHECKPOINT_DIR}")
print(f"   • Outputs: {OUTPUT_DIR}")

## Step 2: Load TTS Model

In [ ]:
from TTS.api import TTS

print("🤖 Loading TTS model...")
print("="*60)

# Note: Using coqui-tts (Idiap fork) for Python 3.12+ compatibility
# The API remains the same as the original Coqui TTS

try:
    # Load pre-trained model (for demo purposes)
    # In production, this would load your fine-tuned checkpoint
    model_name = "tts_models/en/ljspeech/vits"
    
    print(f"Loading model: {model_name}")
    tts_model = TTS(model_name=model_name, progress_bar=True, gpu=torch.cuda.is_available())
    
    print(f"\n✅ Model loaded successfully!")
    print(f"   • Model: VITS (Variational Inference TTS)")
    print(f"   • Language: English")
    print(f"   • Device: {tts_model.device}")
    print(f"   • TTS Library: coqui-tts (Python 3.12+ compatible)")
    
    # For production: Load fine-tuned checkpoint
    # checkpoint_path = f"{CHECKPOINT_DIR}/best/model_best.pth"
    # if os.path.exists(checkpoint_path):
    #     tts_model.load_checkpoint(checkpoint_path)
    #     print(f"   • Loaded fine-tuned checkpoint: {checkpoint_path}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise

print("="*60)

## Step 3: Define Inference Function

In [ ]:
import tempfile
from datetime import datetime

def generate_speech(text, model_choice="Fine-tuned"):
    """
    Generate speech from text using TTS model
    
    Args:
        text (str): Input text to synthesize
        model_choice (str): "Base" or "Fine-tuned" model
    
    Returns:
        str: Path to generated audio file
    """
    try:
        if not text or len(text.strip()) == 0:
            return None
        
        # Limit text length for demo
        if len(text) > 500:
            text = text[:500] + "..."
        
        # Generate unique filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"{OUTPUT_DIR}/generated_{timestamp}.wav"
        
        # Generate speech
        print(f"🎙️ Generating speech for: {text[:50]}...")
        tts_model.tts_to_file(text=text, file_path=output_path)
        
        print(f"✅ Audio generated: {output_path}")
        return output_path
    
    except Exception as e:
        print(f"❌ Error generating speech: {e}")
        return None

# Test the function
test_text = "Hello! This is a demonstration of the voice AI system."
test_output = generate_speech(test_text)

if test_output:
    print(f"\n🎵 Test generation successful!")
    print(f"   Output: {test_output}")
    
    # Display audio player
    import IPython.display as ipd
    display(ipd.Audio(test_output))
else:
    print(f"\n⚠️ Test generation failed")

## Step 4: Create Gradio Interface

In [ ]:
print("🎨 Creating Gradio interface...")
print("="*60)

# Define Gradio interface
def tts_interface(text):
    """Main TTS interface function for Gradio"""
    if not text:
        return None
    
    audio_path = generate_speech(text)
    return audio_path

# Create interface
with gr.Blocks(title="Voice AI - TTS Demo", theme=gr.themes.Soft()) as demo:
    
    gr.Markdown("""
    # 🎙️ Voice AI - Text-to-Speech Demo
    
    Generate realistic human-like speech from text using fine-tuned TTS models.
    
    **Features:**
    - ✅ Natural, expressive voice synthesis
    - ✅ Real-time audio generation
    - ✅ Download generated audio
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            text_input = gr.Textbox(
                label="�� Input Text",
                placeholder="Enter text to synthesize (max 500 characters)...",
                lines=5,
                max_lines=10
            )
            
            # Example texts
            gr.Examples(
                examples=[
                    ["Hello! Welcome to our voice AI demonstration. This system can generate natural-sounding speech from any text."],
                    ["The quick brown fox jumps over the lazy dog. This is a test of expressive text-to-speech synthesis."],
                    ["Good morning! How are you doing today? I hope you're having a wonderful day."],
                    ["Artificial intelligence is transforming the way we interact with technology."],
                    ["This voice AI model has been fine-tuned to produce high-quality, emotional speech."]
                ],
                inputs=text_input,
                label="📋 Example Texts"
            )
            
            with gr.Row():
                clear_btn = gr.Button("🗑️ Clear", variant="secondary")
                generate_btn = gr.Button("🎙️ Generate Speech", variant="primary")
        
        with gr.Column(scale=1):
            audio_output = gr.Audio(
                label="🔊 Generated Audio",
                type="filepath",
                interactive=False
            )
            
            gr.Markdown("""
            ### ℹ️ Instructions:
            1. Enter your text in the input box
            2. Click "Generate Speech"
            3. Listen to the generated audio
            4. Download if needed (click ⋮ menu)
            
            ### 📊 Model Info:
            - Model: VITS (LJSpeech)
            - Language: English
            - Sample Rate: 22,050 Hz
            """)
    
    # Event handlers
    generate_btn.click(
        fn=tts_interface,
        inputs=text_input,
        outputs=audio_output
    )
    
    clear_btn.click(
        fn=lambda: ("", None),
        inputs=None,
        outputs=[text_input, audio_output]
    )

print("✅ Gradio interface created!")
print("="*60)

## Step 5: Launch Gradio Demo

In [ ]:
print("🚀 Launching Gradio demo...")
print("="*60)

# Launch the interface
demo.launch(
    share=True,  # Create public link
    debug=True,
    show_error=True,
    inline=False  # Open in new tab for better experience
)

print("\n✅ Demo launched successfully!")
print("="*60)
print("\n📱 Access your demo:")
print("   • Local URL will appear above")
print("   • Public URL (share=True) will also be generated")
print("   • Share the public URL with others to demo your model")
print("\n⚠️ Note: The Gradio interface will run until you stop this cell")
print("="*60)